In [ ]:
## LEI.py

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import HiveContext, SQLContext,SparkSession
from pyspark.sql import functions as F
from datetime import date,timedelta
from datetime import datetime
sc = SparkContext(appName = "Combined_Table")
sql_context = HiveContext(sc)
spark = SparkSession(sc)
spark = (SparkSession.builder.enableHiveSupport().getOrCreate())


In [ ]:
spark 
import pandas as pd
import os
import numpy as np
import datetime
import time
import glob
import os
from numpy import nan
from pyspark.sql import functions as F
from pyspark.sql.functions import format_string

pd.set_option('display.width', 150)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
exec(open('/home/sharb24/SBB/thural2/record.py').read())


In [ ]:
dteq

In [ ]:
################################### PROCESS LEI #########################################
df1=spark.sql("select max(record_date) as max_rec from anp_cabbtdct1_working.SBB_LEI_F24")
df2 = spark.table('anp_cabbtdct1_working.SBB_LEI_F24')
df3 = df1.join(df2, how='inner', on= F.col('max_rec')==F.col('record_date'))
df_lei=df3.toPandas()

## make lowercase
df_lei.columns=df_lei.columns.str.lower()

## change to float
df_lei = round(df_lei.astype({"plan": float}),2)

## exclude blanks and BBS
df_lei=df_lei[(df_lei.name !='') & (df_lei.name !='BBS')]

#rename the variables
df_lei.rename(columns = {"nov":'november', "dec": 'december',
                       "jan":'january', "feb": 'february',
                       "mar":'march', "apr": 'april',
                       "may":'may', "jun": 'june',
                       "jul":'july', "aug": 'august',
                       "sep":'september', "oct": 'october'}, inplace = True) 

##replace dash (-) with blank
df_lei=df_lei.replace(" -",0.0)
df_lei=df_lei.replace("-",0.0)
df_lei=df_lei.replace("n/a",0.0)

##convert to float
df_lei = round(df_lei.astype({"november": float}),2)
df_lei = round(df_lei.astype({"december": float}),2)
df_lei = round(df_lei.astype({"january": float}),2)
df_lei = round(df_lei.astype({"february": float}),2)
df_lei = round(df_lei.astype({"march": float}),2)
df_lei = round(df_lei.astype({"april": float}),2)
df_lei = round(df_lei.astype({"may": float}),2)
df_lei = round(df_lei.astype({"june": float}),2)
df_lei = round(df_lei.astype({"july": float}),2)
df_lei = round(df_lei.astype({"august": float}),2)
df_lei = round(df_lei.astype({"september": float}),2)
df_lei = round(df_lei.astype({"october": float}),2)
df_lei = round(df_lei.astype({"q1": float}),2)
df_lei = round(df_lei.astype({"q2": float}),2)
df_lei = round(df_lei.astype({"q3": float}),2)
df_lei = round(df_lei.astype({"q4": float}),2)
df_lei = round(df_lei.astype({"ytd": float}),2)

##round all vars
#df_lei['november']=round(df_lei['november'],2)

## order the columns
df_lei=df_lei[['name','cost_centre','november','december','january','q1',
                     'february','march','april','q2','may','june','july','q3',
                     'august','september','october','q4','ytd','plan']]

df_lei['metric_name']='LEI'
df_lei['record_date']=rec_date

In [ ]:
df_lei.head(5)

In [ ]:
################################# GET AMSB Output #################################
###### first, merge am cc to hr/sbb alignment
df=spark.sql("select acf2_id,employee_name,am_cost_center,am_cost_center_name,am_cost_center_full_name, \
                     sm_cost_center,sm_cost_center_name from \
                     (select *, row_number() over(partition by acf2_id order by record_date desc) as ran \
                     from anp_cabbtdct1_final.HRM_SBB_ALIGNMENT0)b where ran=1")
dfhr=df.toPandas()

### (1) exclude RAMSBs
dfhr1=dfhr[(dfhr.am_cost_center!='3838') & (dfhr.am_cost_center!='5311')]

lei_new4=pd.merge(dfhr1,df_lei,how='inner',left_on=['am_cost_center'],right_on=['cost_centre'])


## order the vars
amsb=lei_new4[['metric_name','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q2','q3','q4','ytd','plan','am_cost_center','am_cost_center_name','am_cost_center_full_name',
           'sm_cost_center','sm_cost_center_name','record_date']]

## (2) get only RAMSBs
dfhr2=dfhr[(dfhr.am_cost_center=='3838') | (dfhr.am_cost_center=='5311')]
dfhr2['name']='RAMP'

lei_new44=pd.merge(dfhr2,df_lei,how='inner',on=['name'])

ramsb=lei_new44[['metric_name','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q2','q3','q4','ytd','plan','am_cost_center','am_cost_center_name','am_cost_center_full_name',
           'sm_cost_center','sm_cost_center_name','record_date']]

## concatenate the AMSB and RAMSB
all_amsb=pd.concat([ramsb,amsb],axis=0)


In [ ]:
############################### PROCESS AMSB/RAMSB #############################################

lei_new5=all_amsb.copy()
lei_new5['level']='AMSB'
lei_new5['scorecard_filter']='SBB'
lei_new5['record_date']=rec_date

lei_amsb=lei_new5[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q2','q3','q4','ytd','plan','am_cost_center','am_cost_center_name','am_cost_center_full_name',
           'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']].drop_duplicates()

### blank out quarter and ytd benchmarks
lei_amsb['q1_benchmark']=float("NaN")
lei_amsb['q2_benchmark']=float("NaN")
lei_amsb['q3_benchmark']=float("NaN")
lei_amsb['q4_benchmark']=float("NaN")
lei_amsb['ytd_benchmark']=float("NaN")

## YTD PLAN IS ALSO QTD PLAN
lei_amsb['q1_plan']=lei_amsb.plan
lei_amsb['q2_plan']=lei_amsb.plan
lei_amsb['q3_plan']=lei_amsb.plan
lei_amsb['q4_plan']=lei_amsb.plan

### CALCULATE PACE TO PLAN
lei_amsb['q1_pace_to_plan']=((lei_amsb.q1.fillna(0))*q1_tot/q1_passed)/(lei_amsb.plan)
lei_amsb['q2_pace_to_plan']=((lei_amsb.q2.fillna(0))*q1_tot/q2_passed)/(lei_amsb.plan)
lei_amsb['q3_pace_to_plan']=((lei_amsb.q3.fillna(0))*q1_tot/q3_passed)/(lei_amsb.plan)
lei_amsb['q4_pace_to_plan']=((lei_amsb.q4.fillna(0))*q1_tot/q4_passed)/(lei_amsb.plan)

### order variables
lei_amsb=lei_amsb[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q1_plan','q1_pace_to_plan','q2','q2_benchmark','q2_plan','q2_pace_to_plan',
           'q3','q3_benchmark','q3_plan','q3_pace_to_plan','q4','q4_benchmark','q4_plan',
           'q4_pace_to_plan','ytd','ytd_benchmark',
           'am_cost_center','am_cost_center_name','am_cost_center_full_name',
           'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
lei_amsb.head(4)

In [ ]:
############################# CREATE THE AM OUTPUT #############################################

lei_am=lei_amsb[['metric_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q1_plan','q1_pace_to_plan','q2','q2_benchmark','q2_plan','q2_pace_to_plan',
           'q3','q3_benchmark','q3_plan','q3_pace_to_plan','q4','q4_benchmark','q4_plan',
           'q4_pace_to_plan','ytd','ytd_benchmark',
            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
           'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']].drop_duplicates()

lei_am['acf2_id']=''
lei_am['employee_name']=lei_am.am_cost_center_full_name
lei_am['scorecard_filter']='Total'
lei_am['level']='AM'

lei_am1=lei_am.copy()
lei_am1['scorecard_filter']='SBB'

lei_am2=lei_am1.copy()
lei_am2['scorecard_filter']='Retail'

##order the variables and concatenate
lei_am=lei_am[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q1_plan','q1_pace_to_plan','q2','q2_benchmark','q2_plan','q2_pace_to_plan',
           'q3','q3_benchmark','q3_plan','q3_pace_to_plan','q4','q4_benchmark','q4_plan',
           'q4_pace_to_plan','ytd','ytd_benchmark',
           'am_cost_center','am_cost_center_name','am_cost_center_full_name',
           'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

lei_am1=lei_am1[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q1_plan','q1_pace_to_plan','q2','q2_benchmark','q2_plan','q2_pace_to_plan',
           'q3','q3_benchmark','q3_plan','q3_pace_to_plan','q4','q4_benchmark','q4_plan',
           'q4_pace_to_plan','ytd','ytd_benchmark',
           'am_cost_center','am_cost_center_name','am_cost_center_full_name',
           'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

lei_am2=lei_am2[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q1_plan','q1_pace_to_plan','q2','q2_benchmark','q2_plan','q2_pace_to_plan',
           'q3','q3_benchmark','q3_plan','q3_pace_to_plan','q4','q4_benchmark','q4_plan',
           'q4_pace_to_plan','ytd','ytd_benchmark',
           'am_cost_center','am_cost_center_name','am_cost_center_full_name',
           'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

## concatenate
all_am=pd.concat([lei_am,lei_am1,lei_am2],axis=0)


In [ ]:
############################# CREATE THE SM OUTPUT ###########################################

## get unique sm cc from alignment
lei_sm=lei_amsb[['sm_cost_center','sm_cost_center_name']].drop_duplicates()

## make distinct 
df_leik=df_lei.drop(columns='name')
df_leik1=df_leik.drop_duplicates()

## merge to LEI data
lei_sm1=pd.merge(lei_sm,df_leik1,how='inner',left_on=['sm_cost_center'],right_on=['cost_centre'])

lei_sm1['acf2_id']=''
lei_sm1['employee_name']=lei_sm1.sm_cost_center_name
lei_sm1['scorecard_filter']='Total'
lei_sm1['level']='SM'
lei_sm1['am_cost_center']=''
lei_sm1['am_cost_center_name']=''
lei_sm1['am_cost_center_full_name']=''
lei_sm1['record_date']=rec_date

### blank out quarter and ytd benchmarks
lei_sm1['q1_benchmark']=float("NaN")
lei_sm1['q2_benchmark']=float("NaN")
lei_sm1['q3_benchmark']=float("NaN")
lei_sm1['q4_benchmark']=float("NaN")
lei_sm1['ytd_benchmark']=float("NaN")

## YTD PLAN IS ALSO QTD PLAN
lei_sm1['q1_plan']=lei_sm1.plan
lei_sm1['q2_plan']=lei_sm1.plan
lei_sm1['q3_plan']=lei_sm1.plan
lei_sm1['q4_plan']=lei_sm1.plan

### CALCULATE PACE TO PLAN
lei_sm1['q1_pace_to_plan']=((lei_sm1.q1.fillna(0))*q1_tot/q1_passed)/(lei_sm1.plan)
lei_sm1['q2_pace_to_plan']=((lei_sm1.q2.fillna(0))*q1_tot/q2_passed)/(lei_sm1.plan)
lei_sm1['q3_pace_to_plan']=((lei_sm1.q3.fillna(0))*q1_tot/q3_passed)/(lei_sm1.plan)
lei_sm1['q4_pace_to_plan']=((lei_sm1.q4.fillna(0))*q1_tot/q4_passed)/(lei_sm1.plan)

lei_sm2=lei_sm1[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q1_plan','q1_pace_to_plan','q2','q2_benchmark','q2_plan','q2_pace_to_plan',
           'q3','q3_benchmark','q3_plan','q3_pace_to_plan','q4','q4_benchmark','q4_plan',
           'q4_pace_to_plan','ytd','ytd_benchmark',
           'am_cost_center','am_cost_center_name','am_cost_center_full_name',
           'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

## create sbb and retail scorecard_filter
lei_sm3=lei_sm2.copy()
lei_sm3['scorecard_filter']='SBB'

lei_sm3=lei_sm3[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q1_plan','q1_pace_to_plan','q2','q2_benchmark','q2_plan','q2_pace_to_plan',
           'q3','q3_benchmark','q3_plan','q3_pace_to_plan','q4','q4_benchmark','q4_plan',
           'q4_pace_to_plan','ytd','ytd_benchmark',
           'am_cost_center','am_cost_center_name','am_cost_center_full_name',
           'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

lei_sm4=lei_sm2.copy()
lei_sm4['scorecard_filter']='Retail'

lei_sm4=lei_sm4[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q1_plan','q1_pace_to_plan','q2','q2_benchmark','q2_plan','q2_pace_to_plan',
           'q3','q3_benchmark','q3_plan','q3_pace_to_plan','q4','q4_benchmark','q4_plan',
           'q4_pace_to_plan','ytd','ytd_benchmark',
           'am_cost_center','am_cost_center_name','am_cost_center_full_name',
           'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

## concatenate the dataframes
all_sm=pd.concat([lei_sm2,lei_sm3,lei_sm4],axis=0)


In [ ]:
lei_sm1

In [ ]:
############################# CREATE THE NATIONAL OUTPUT ###########################################

## get National LEI score
lei_nso=df_lei[(df_lei.name.str.contains('National'))]

lei_nso['acf2_id']=''
lei_nso['employee_name']='National Office'
lei_nso['scorecard_filter']='Total'
lei_nso['level']='National Office'
lei_nso['am_cost_center']=''
lei_nso['am_cost_center_name']=''
lei_nso['am_cost_center_full_name']=''
lei_nso['sm_cost_center']=''
lei_nso['sm_cost_center_name']=''

## YTD PLAN IS ALSO QTD PLAN
lei_nso['q1_plan']=lei_nso.plan
lei_nso['q2_plan']=lei_nso.plan
lei_nso['q3_plan']=lei_nso.plan
lei_nso['q4_plan']=lei_nso.plan

### CALCULATE PACE TO PLAN
lei_nso['q1_pace_to_plan']=((lei_nso.q1.fillna(0))*q1_tot/q1_passed)/(lei_nso.plan)
lei_nso['q2_pace_to_plan']=((lei_nso.q2.fillna(0))*q1_tot/q2_passed)/(lei_nso.plan)
lei_nso['q3_pace_to_plan']=((lei_nso.q3.fillna(0))*q1_tot/q3_passed)/(lei_nso.plan)
lei_nso['q4_pace_to_plan']=((lei_nso.q4.fillna(0))*q1_tot/q4_passed)/(lei_nso.plan)

### blank out quarter and ytd benchmarks
lei_nso['q1_benchmark']=float("NaN")
lei_nso['q2_benchmark']=float("NaN")
lei_nso['q3_benchmark']=float("NaN")
lei_nso['q4_benchmark']=float("NaN")
lei_nso['ytd_benchmark']=float("NaN")


lei_nso2=lei_nso[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q1_plan','q1_pace_to_plan','q2','q2_benchmark','q2_plan','q2_pace_to_plan',
           'q3','q3_benchmark','q3_plan','q3_pace_to_plan','q4','q4_benchmark','q4_plan',
           'q4_pace_to_plan','ytd','ytd_benchmark',
           'am_cost_center','am_cost_center_name','am_cost_center_full_name',
           'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

######################## concatenate AMSB,AM,SM,NSO and save as table#################################

all_lei=pd.concat([lei_amsb,all_am,all_sm,lei_nso2],axis=0)


In [ ]:
all_lei=all_lei.replace(np.nan,0.0)

In [ ]:
all_lei.november.unique()

In [ ]:
#from pyspark.sql.types import *
#create schema for your dataframe
#schema = StructType([StructField("metric_name", StringType(), True)\
#                   ,StructField("level", StringType(), True)\
#                   ,StructField("acf2_id", StringType(), True)\
#                     ,StructField("employee_name", StringType(), True)\
#                     ,StructField("november", FloatType(), True)\
#                     ,StructField("december", FloatType(), True)\
#                     ,StructField("january", FloatType(), True)\
#                     ,StructField("february", FloatType(), True)\
#                     ,StructField("march", FloatType(), True)\
#                     ,StructField("april", FloatType(), True)\
#                     ,StructField("may", FloatType(), True)\
#                     ,StructField("june", FloatType(), True)\
#                     ,StructField("july", FloatType(), True)\
#                     ,StructField("august", FloatType(), True)\
#                     ,StructField("september", FloatType(), True)\
#                     ,StructField("october", FloatType(), True)\
#                     ,StructField("q1", FloatType(), True)\
#                     ,StructField("q1_benchmark", FloatType(), True)\
#                     ,StructField("q1_plan", FloatType(), True)\
#                     ,StructField("q1_pace_to_plan", FloatType(), True)\
#                     ,StructField("q2", FloatType(), True)\
#                     ,StructField("q2_benchmark", FloatType(), True)\
#                     ,StructField("q2_plan", FloatType(), True)\
#                     ,StructField("q2_pace_to_plan", FloatType(), True)\
#                     ,StructField("q3", FloatType(), True)\
#                     ,StructField("q3_benchmark", FloatType(), True)\
#                     ,StructField("q3_plan", FloatType(), True)\
#                     ,StructField("q3_pace_to_plan", FloatType(), True)\
#                    ,StructField("q4", FloatType(), True)\
#                     ,StructField("q4_benchmark", FloatType(), True)\
#                     ,StructField("q4_plan", FloatType(), True)\
#                     ,StructField("q4_pace_to_plan", FloatType(), True)\
#                     ,StructField("ytd", FloatType(), True)\
#                     ,StructField("ytd_benchmark", FloatType(), True)\
#                     ,StructField("am_cost_center", StringType(), True)\
#                     ,StructField("am_cost_center_name", StringType(), True)\
#                     ,StructField("am_cost_center_full_name", StringType(), True)\
#                     ,StructField("sm_cost_center", StringType(), True)\
#                      ,StructField("sm_cost_center_name", StringType(), True)\
#                      ,StructField("scorecard_filter", StringType(), True)\
#                      ,StructField("record_date", StringType(), True)])

#create spark dataframe using schema
#all_leifinal = spark.createDataFrame(all_lei,schema=schema)

In [ ]:
# Remove all values for the month of November
all_lei.head(2)


In [ ]:
## save in table
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.createDataFrame(all_lei).write.mode("append").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_OUTPUT_LEI")


In [ ]:
## END OF PROGRAM


In [ ]:
#all_lei['employee_name'] = all_lei['employee_name'].astype('str') 

In [ ]:
#all_lei.head(5)